In [99]:
import mpilot
import arcpy
import inspect
import pandas as pd
import geopandas as gpd
import os
import shutil
from shapely import wkt
from pyproj import CRS

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

In [100]:
arcpy.ClearWorkspaceCache_management()

<Result 'true'>

In [102]:
arcpy.ResetEnvironments()
gdb_name = "Ember_Risk.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
# arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

In [103]:
path = os.path.join(THIS_FOLDER, "out_data/ru/Reporting_Units_APN.shp")
arcpy.CopyFeatures_management(path, "Reporting_Units")
crs = arcpy.SpatialReference('WGS 1984')
arcpy.Project_management("Reporting_Units", "Reporting_Units_rpj", crs)

path = os.path.join(THIS_FOLDER, "in_data/Prob_Lg_Fire_Vector.shp")
arcpy.CopyFeatures_management(path, "Prob_Lg_Fire")
crs = arcpy.SpatialReference('WGS 1984')
arcpy.Project_management("Prob_Lg_Fire", "Prob_Lg_Fire_rpj", crs)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\Prob_Lg_Fire_rpj'>

In [104]:
arcpy.management.FeatureToPoint("Reporting_Units_rpj", "Reporting_Units_Points", "INSIDE")     

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\Reporting_Units_Points'>

In [105]:
path = os.path.join(THIS_FOLDER, "in_data/wind/santa_ana_10162018_speed.tif")
arcpy.sa.ExtractValuesToPoints("Reporting_Units_Points", path, "wind_pts1", "NONE", "VALUE_ONLY")
arcpy.AlterField_management('wind_pts1', 'RASTERVALU', 'sa_mag', 'sa_mag')

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\wind_pts1'>

In [106]:
path = os.path.join(THIS_FOLDER, "in_data/wind/santa_ana_10162018_dir.tif")
arcpy.sa.ExtractValuesToPoints("wind_pts1", path, "wind_pts2", "NONE", "VALUE_ONLY")
arcpy.AlterField_management('wind_pts2', 'RASTERVALU', 'sa_dir', 'sa_dir')

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\wind_pts2'>

In [107]:
path = os.path.join(THIS_FOLDER, "in_data/wind/sundowner_4132018_dir.tif")
arcpy.sa.ExtractValuesToPoints("wind_pts2", path, "wind_pts3", "NONE", "VALUE_ONLY")
arcpy.AlterField_management('wind_pts3', 'RASTERVALU', 'sd_dir', 'sd_dir')

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\wind_pts3'>

In [108]:
path = os.path.join(THIS_FOLDER, "in_data/wind/sundowner_4132018_speed.tif")
arcpy.sa.ExtractValuesToPoints("wind_pts3", path, "wind_pts4", "NONE", "VALUE_ONLY")
arcpy.AlterField_management('wind_pts4', 'RASTERVALU', 'sd_mag', 'sd_mag')

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\wind_pts4'>

In [109]:
arcpy.AddField_management("wind_pts4", "x", "DOUBLE")
arcpy.AddField_management("wind_pts4", "y", "DOUBLE")
arcpy.management.CalculateGeometryAttributes("wind_pts4", "x POINT_X;y POINT_Y", '', '', None, "SAME_AS_INPUT")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\wind_pts4'>

In [110]:
arcpy.AddField_management("wind_pts4", "sa_throw", "DOUBLE")
arcpy.AddField_management("wind_pts4", "sd_throw", "DOUBLE")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\wind_pts4'>

In [111]:
codeblock = """
def get_throw(mag):
    return (mag * 100)
"""

# Execute CalculateField 
arcpy.CalculateField_management("wind_pts4", "sa_throw", "get_throw(!sa_mag!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("wind_pts4", "sd_throw", "get_throw(!sd_mag!)", "PYTHON3", codeblock)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\wind_pts4'>

In [112]:
fieldNames = ["sa_throw", "sd_throw"] 
fieldCount = len(fieldNames)

with arcpy.da.UpdateCursor("wind_pts4", fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if (rowU[field] == None or rowU[field] == 0):  
                rowU[field] = "0.01"  
        curU.updateRow(rowU)

del curU

In [113]:
arcpy.management.BearingDistanceToLine("wind_pts4", "sd_bearing_line", "x", "y", "sd_throw", "METERS", "sd_dir", "DEGREES", "GEODESIC", None)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\sd_bearing_line'>

In [114]:
arcpy.management.BearingDistanceToLine("wind_pts4", "sa_bearing_line", "x", "y", "sa_throw", "METERS", "sa_dir", "DEGREES", "GEODESIC", None)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\sa_bearing_line'>

In [146]:
field_names = [f.name for f in arcpy.ListFields("sa_bearing_line")]
print(field_names)

['OID', 'Shape', 'x', 'y', 'sa_throw', 'sa_dir', 'Shape_Length']


In [147]:
field_names = [f.name for f in arcpy.ListFields("sd_bearing_line")]
print(field_names)

['OID', 'Shape', 'x', 'y', 'sd_throw', 'sd_dir', 'Shape_Length']


In [148]:
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable("sa_bearing_line")
fieldmappings.addTable("Prob_Lg_Fire_rpj")
keepers = ["OID", "mean_sa_throw"]

pcFieldIndex = fieldmappings.findFieldMapIndex("ProbLgFire")
pcfieldmap = fieldmappings.getFieldMap(pcFieldIndex)
pcnew = pcfieldmap.outputField
pcnew.name = "mean_sa_throw"
pcnew.aliasName = "mean_sa_throw"
pcfieldmap.outputField = pcnew
pcfieldmap.mergeRule = "mean"

fieldmappings.replaceFieldMap(pcFieldIndex, pcfieldmap)

for field in fieldmappings.fields:
    if field.name not in keepers:
        fieldmappings.removeFieldMap(fieldmappings.findFieldMapIndex(field.name))
        
arcpy.SpatialJoin_analysis("sa_bearing_line", "Prob_Lg_Fire_rpj", "sa_bearing_line_join", "JOIN_ONE_TO_ONE", "#", fieldmappings) 

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\sa_bearing_line_join'>

In [149]:
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable("sd_bearing_line")
fieldmappings.addTable("Prob_Lg_Fire_rpj")
keepers = ["OID", "mean_sd_throw"]

pcFieldIndex = fieldmappings.findFieldMapIndex("ProbLgFire")
pcfieldmap = fieldmappings.getFieldMap(pcFieldIndex)
pcnew = pcfieldmap.outputField
pcnew.name = "mean_sd_throw"
pcnew.aliasName = "mean_sd_throw"
pcfieldmap.outputField = pcnew
pcfieldmap.mergeRule = "mean"

fieldmappings.replaceFieldMap(pcFieldIndex, pcfieldmap)

for field in fieldmappings.fields:
    if field.name not in keepers:
        fieldmappings.removeFieldMap(fieldmappings.findFieldMapIndex(field.name))
        
arcpy.SpatialJoin_analysis("sd_bearing_line", "Prob_Lg_Fire_rpj", "sd_bearing_line_join", "JOIN_ONE_TO_ONE", "#", fieldmappings) 

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\sd_bearing_line_join'>

In [150]:
field_names = [f.name for f in arcpy.ListFields("sd_bearing_line_join")]
print(field_names)

['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'mean_sd_throw', 'Shape_Length']


In [151]:
# ember_risk = 'mean_sd_throw' * 'Shape_Length'
arcpy.AddField_management("sd_bearing_line_join", "sd_ember_risk", "DOUBLE")
arcpy.AddField_management("sa_bearing_line_join", "sa_ember_risk", "DOUBLE")

codeblock = """
def get_ember_risk(risk, l):
    return (risk * l)
"""

# Execute CalculateField 
arcpy.CalculateField_management("sd_bearing_line_join", "sd_ember_risk", "get_ember_risk(!mean_sd_throw!, !Shape_Length!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("sa_bearing_line_join", "sa_ember_risk", "get_ember_risk(!mean_sa_throw!, !Shape_Length!)", "PYTHON3", codeblock)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\sa_bearing_line_join'>

In [152]:
joined_table = arcpy.management.AddJoin("wind_pts4", "OBJECTID", "sd_bearing_line_join", "OBJECTID", "KEEP_ALL")
arcpy.CopyFeatures_management(joined_table, "sd_ember")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\sd_ember'>

In [153]:
joined_table = arcpy.management.AddJoin("wind_pts4", "OBJECTID", "sa_bearing_line_join", "OBJECTID", "KEEP_ALL")
arcpy.CopyFeatures_management(joined_table, "sa_ember")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\sa_ember'>

In [154]:
joined_table = arcpy.management.AddJoin("sa_ember", "OBJECTID", "sd_ember", "OBJECTID", "KEEP_ALL")
arcpy.CopyFeatures_management(joined_table, "ember_risk")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\ember_risk'>

In [155]:
field_names = [f.name for f in arcpy.ListFields("ember_risk")]
print(field_names)

['OBJECTID', 'Shape', 'sa_ember_wind_pts4_Join_ID', 'sa_ember_wind_pts4_apn', 'sa_ember_wind_pts4_Shape_Leng', 'sa_ember_wind_pts4_ORIG_FID', 'sa_ember_wind_pts4_sa_mag', 'sa_ember_wind_pts4_sa_dir', 'sa_ember_wind_pts4_sd_dir', 'sa_ember_wind_pts4_sd_mag', 'sa_ember_wind_pts4_x', 'sa_ember_wind_pts4_y', 'sa_ember_wind_pts4_sa_throw', 'sa_ember_wind_pts4_sd_throw', 'sa_ember_sa_bearing_line_join_OBJECTID', 'sa_ember_sa_bearing_line_join_Join_Count', 'sa_ember_sa_bearing_line_join_TARGET_FID', 'sa_ember_sa_bearing_line_join_mean_sa_throw', 'sa_ember_sa_bearing_line_join_sa_ember_risk', 'sd_ember_OBJECTID', 'sd_ember_wind_pts4_Join_ID', 'sd_ember_wind_pts4_apn', 'sd_ember_wind_pts4_Shape_Leng', 'sd_ember_wind_pts4_ORIG_FID', 'sd_ember_wind_pts4_sa_mag', 'sd_ember_wind_pts4_sa_dir', 'sd_ember_wind_pts4_sd_dir', 'sd_ember_wind_pts4_sd_mag', 'sd_ember_wind_pts4_x', 'sd_ember_wind_pts4_y', 'sd_ember_wind_pts4_sa_throw', 'sd_ember_wind_pts4_sd_throw', 'sd_ember_sd_bearing_line_join_OBJECTID',

In [157]:
# if you can use a key to identify the fields to remove, then it's solved
fields = arcpy.ListFields("ember_risk") 

# manually enter field names to keep here
# include mandatory fields name such as OBJECTID (or FID), and Shape in keepfields
keepFields = ['OBJECTID', 'Shape', 
              'sa_ember_wind_pts4_sa_mag', 'sa_ember_wind_pts4_sa_dir', 'sa_ember_wind_pts4_sd_dir', 'sa_ember_wind_pts4_sd_mag',
              'sd_ember_sd_bearing_line_join_mean_sd_throw', 'sa_ember_sa_bearing_line_join_mean_sa_throw',
              'sa_ember_sa_bearing_line_join_sa_ember_risk', 'sd_ember_sd_bearing_line_join_sd_ember_risk']

dropFields = [x.name for x in fields if x.name not in keepFields]
# delete fields
arcpy.DeleteField_management("ember_risk", dropFields)   

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\ember_risk'>

In [158]:
field_names = [f.name for f in arcpy.ListFields("ember_risk")]
print(field_names)

['OBJECTID', 'Shape', 'sa_ember_wind_pts4_sa_mag', 'sa_ember_wind_pts4_sa_dir', 'sa_ember_wind_pts4_sd_dir', 'sa_ember_wind_pts4_sd_mag', 'sa_ember_sa_bearing_line_join_mean_sa_throw', 'sa_ember_sa_bearing_line_join_sa_ember_risk', 'sd_ember_sd_bearing_line_join_mean_sd_throw', 'sd_ember_sd_bearing_line_join_sd_ember_risk']


In [159]:
arcpy.AlterField_management("ember_risk", 
                            'sa_ember_wind_pts4_sa_mag', 'sa_mag', 'sa_mag')
arcpy.AlterField_management("ember_risk", 
                            'sa_ember_wind_pts4_sa_dir', 'sa_dir', 'sa_dir')
arcpy.AlterField_management("ember_risk", 
                            'sa_ember_wind_pts4_sd_dir', 'sd_dir', 'sd_dir')
arcpy.AlterField_management("ember_risk", 
                            'sa_ember_wind_pts4_sd_mag', 'sd_mag', 'sd_mag')
arcpy.AlterField_management("ember_risk", 
                            'sa_ember_sa_bearing_line_join_mean_sa_throw', 'av_sa_rsk', 'av_sa_rsk')
arcpy.AlterField_management("ember_risk", 
                            'sd_ember_sd_bearing_line_join_mean_sd_throw', 'av_sd_rsk', 'av_sd_rsk')
arcpy.AlterField_management("ember_risk", 
                            'sa_ember_sa_bearing_line_join_sa_ember_risk', 'sa_emb_rsk', 'sa_emb_rsk')
arcpy.AlterField_management("ember_risk", 
                            'sd_ember_sd_bearing_line_join_sd_ember_risk', 'sd_emb_rsk', 'sd_emb_rsk')

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Ember_Risk.gdb\\ember_risk'>

In [162]:
field_names = [f.name for f in arcpy.ListFields("ember_risk")]
print(field_names)

['OBJECTID', 'Shape', 'sa_mag', 'sa_dir', 'sd_dir', 'sd_mag', 'av_sa_rsk', 'sa_emb_rsk', 'av_sd_rsk', 'sd_emb_rsk']


In [164]:
dir_path = os.path.join(THIS_FOLDER, "out_data/ember")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

arcpy.conversion.FeatureClassToShapefile("ember_risk", 
                                        os.path.join(THIS_FOLDER, "out_data/ember"))

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\out_data\\ember'>